In [1]:
import numpy as np
import pandas as pd
import re
from pprint import pprint
raw_df = pd.read_excel("测序数据统计-早筛组 20190211-20200207实验记录存档.20200427.xlsx", sheet_name="测序信息登记")
capture_df = pd.read_excel("pooling.20200428.xlsx", sheet_name="capture")


In [2]:
# raw_df列切片去重，对日期进行处理
raw_df_filt = raw_df.loc[:, ["Pooling号", "送测日期", "上机日期", "下机日期", "机器号", "芯片号"]]
tmp = [ x if pd.isna(x) else str(int(x)) for x in raw_df_filt.loc[:,"送测日期"].to_list()]
raw_df_filt["送测日期"] = [ x if pd.isna(x) else x[:4]+"/"+x[4:6]+"/"+x[6:] for x in  tmp]
raw_df_filt["上机日期"] = raw_df_filt["上机日期"].apply(lambda x: "" if x=="无" or pd.isna(x) else pd.to_datetime(x).strftime('%Y/%m/%d'))
raw_df_filt["下机日期"] = raw_df_filt["下机日期"].apply(lambda x: "" if x=="无" or pd.isna(x) else pd.to_datetime(x).strftime('%Y/%m/%d'))
raw_df_filt = raw_df_filt.fillna(method='ffill')
raw_df_filt = raw_df_filt.drop_duplicates()
raw_df_filt.shape

(795, 6)

In [4]:
#根据Pooling号合并raw_df_filt和capture_df
raw_df_filt.loc[:, "Pooling号"] = raw_df_filt.loc[:, "Pooling号"].str.strip()
capture_df.loc[:, "虚拟入库样本名"] = capture_df.loc[:, "虚拟入库样本名"].str.strip()
merge_df = pd.merge(raw_df_filt, capture_df, left_on="Pooling号", right_on="虚拟入库样本名")
merge_df.columns = ['虚拟入库样本名1', '送测日期', '上机时间', '下机时间', '机器号', '芯片号', '捕获文库名', '虚拟入库样本名2',
       '杂交日期', '杂交探针', '杂交时间h', 'PostPCR循环数', 'PostPCR浓度ng/ul', '洗脱体积', '操作人',
       '备注']
merge_df_filt = merge_df.loc[:,['虚拟入库样本名1', '送测日期', '上机时间', '下机时间', '机器号', '芯片号', '捕获文库名']]
# merge_df_filt.head

<bound method NDFrame.head of     虚拟入库样本名1        送测日期        上机时间        下机时间                          机器号  \
0    IDTL130         NaN  2018/12/27  2018/12/29                R100400180038   
1    IDTL131         NaN  2018/12/27  2018/12/29                R100400180038   
2    IDTL142         NaN  2019/01/12  2019/01/14                R100400180015   
3    IDTL145         NaN  2019/01/12  2019/01/14                 100400180007   
4      LM001         NaN  2019/01/16  2019/01/18                R100400180015   
..       ...         ...         ...         ...                          ...   
206    LM285  2020/02/03  2020/02/04  2020/02/06  100400180014  R100400180013   
207    LM286  2020/02/04  2020/02/05  2020/02/07                R100400180038   
208    LM287  2020/02/04  2020/02/05  2020/02/07                R100400180120   
209    LM287  2020/02/04  2020/02/05  2020/02/07                R100400180120   
210    LM289  2020/02/04  2020/02/05  2020/02/07                R100400180025  

In [7]:
#列“机器号”进行处理
tmp = []
for x in merge_df_filt.loc[:, "机器号"]:
    x = re.sub(r'R', "", str(x).strip())
    x = re.sub(r'(\d+)', r'R\1', str(x))
    x = re.sub(r'\s+', r', ', str(x))
    tmp.append(x)
merge_df_filt.loc[:, "机器号"] = tmp


In [17]:
count = {}
tmp = []
for idx in range(merge_df_filt.shape[0]):
    k = re.sub('/', "", merge_df_filt.loc[idx, "上机时间"])
    k = "seq_"+k
    if k in count:
        if merge_df_filt.loc[idx, "机器号"] not in count[k]:
            count[k].append(merge_df_filt.loc[idx, "机器号"])

    else:
        count[k]=[merge_df_filt.loc[idx, "机器号"]]

    
    idx2 = count[k].index(merge_df_filt.loc[idx, "机器号"])
    v = "{}_{}".format(k, idx2+1)
    tmp.append(v)

merge_df_filt.loc[:, "测序编号"] = tmp

In [18]:
merge_df_filt

,虚拟入库样本名1,送测日期,上机时间,下机时间,机器号,芯片号,捕获文库名,测序编号
0,IDTL130,NaN,2018/12/27,2018/12/29,R100400180038,"V300012489, V300012514",ssM-CpGiant-1-20181218,seq_20181227_1
1,IDTL131,NaN,2018/12/27,2018/12/29,R100400180038,"V300012489, V300012514",ssM-CpGiant-2-20181218,seq_20181227_1
2,IDTL142,NaN,2019/01/12,2019/01/14,R100400180015,"V300012566, V300012600,",ssM-CpGiant-1-20190104,seq_20190112_1
3,IDTL145,NaN,2019/01/12,2019/01/14,R100400180007,"V300012612, V300012618",ssM-CpGiant-4-20190104,seq_20190112_2
4,LM001,NaN,2019/01/16,2019/01/18,R100400180015,"V300012535, V300012554",ssM-CpGiant-2-20190109,seq_20190116_1
...,...,...,...,...,...,...,...,...
206,LM285,2020/02/03,2020/02/04,2020/02/06,"R100400180014, R100400180013","V300032757, V300032653",ssM-CpGiant-4-20200131,seq_20200204_2
207,LM286,2020/02/04,2020/02/05,2020/02/07,R100400180038,"V300044882, V300046512",ssM-CpGiant-5-20200131,seq_20200205_1
208,LM287,2020/02/04,2020/02/05,2020/02/07,R100400180120,"V300044876, V300044921",ssM-CpGiant-6-20200131,seq_20200205_2
209,LM287,2020/02/04,2020/02/05,2020/02/07,R100400180120,"V300044876, V300044921",ssM-CpGiant-7-20200131,seq_20200205_2


In [24]:
tmp_dict = {}
for col in merge_df_filt.columns:
    tmp_dict[col] = []

for idx in range(merge_df_filt.shape[0]):
    try:
        idx2 = tmp_dict["测序编号"].index(merge_df_filt.loc[idx, "测序编号"])
        tmp_dict["捕获文库名"][idx2] = tmp_dict["捕获文库名"][idx2] + ", " + merge_df_filt.loc[idx, "捕获文库名"]
        tmp_dict["虚拟入库样本名1"][idx2] = tmp_dict["虚拟入库样本名1"][idx2] + ", " + merge_df_filt.loc[idx, "虚拟入库样本名1"]
    except ValueError:
        for col in merge_df_filt.columns:
            tmp_dict[col].append(merge_df_filt.loc[idx, col])

for idx in range(len(tmp_dict["虚拟入库样本名1"])):
    id_list = tmp_dict["虚拟入库样本名1"][idx].split(", ")
    if len(id_list)>1:
        tmp_dict["虚拟入库样本名1"][idx]=", ".join(list(set(id_list)))


In [26]:
pd.DataFrame(tmp_dict).to_excel("sequence.20200428.xlsx", sheet_name="测序信息表", index=False)

In [32]:
pd.to_datetime(merge_df_filt.loc[0,"上机时间"], format='%Y/%m/%d')

Timestamp('2018-12-27 11:32:00')

In [35]:
merge_df_filt["上机时间"].apply(lambda x: pd.to_datetime(x).strftime('seq_%Y%m%d'))

0      seq_20181227
1      seq_20181227
2      seq_20190112
3      seq_20190112
4      seq_20190116
           ...     
206    seq_20200204
207    seq_20200205
208    seq_20200205
209    seq_20200205
210    seq_20200205
Name: 上机时间, Length: 211, dtype: object

In [20]:
merge_df.columns

Index(['Pooling号', '送测日期', '上机日期', '下机日期', '机器号', '芯片号', '文库名称', '虚拟入库样本名',
       '杂交日期', '杂交探针', '杂交时间h', 'PostPCR循环数', 'PostPCR浓度ng/ul', '洗脱体积', '操作人',
       '备注'],
      dtype='object')

In [8]:
raw_df_filt.head

<bound method NDFrame.head of                     Pooling号        送测日期                 上机日期  \
0      BGISEQ-50018SZ0002233         NaN                    无   
4      MGI2000-ZJC&SLL0510-1         NaN  2018-05-12 00:00:00   
12    BGISEQ-50018SZ0002000          NaN                    无   
15     BGISEQ-50018SZ0002452         NaN                    无   
17     BGISEQ-50018SZ0002806         NaN                    无   
...                      ...         ...                  ...   
7021                   LS302  20200207.0  2020-02-05 15:48:00   
7029                   LS303  20200207.0  2020-02-05 15:48:00   
7037                   LS304  20200207.0  2020-02-05 15:48:00   
7045                   LS305  20200207.0  2020-02-05 15:48:00   
7053                   LS306  20200207.0  2020-02-05 15:48:00   

                     下机日期            机器号                     芯片号  
0                       无              无                       无  
4     2018-05-16 00:00:00  MGISEQ200016               V

In [9]:
capture_df.head

<bound method NDFrame.head of                        文库名称  虚拟入库样本名        杂交日期  \
0    ssM-CpGiant-1-20181218  IDTL130  2018/12/18   
1    ssM-CpGiant-2-20181218  IDTL131  2018/12/18   
2    ssM-CpGiant-1-20190104  IDTL142  2019/01/04   
3    ssM-CpGiant-4-20190104  IDTL145  2019/01/04   
4    ssM-CpGiant-2-20190109    LM001  2019/01/09   
..                      ...      ...         ...   
212  ssM-CpGiant-2-20200214    LM306  2020/02/14   
213  ssM-CpGiant-3-20200214    LM307  2020/02/14   
214  ssM-CpGiant-4-20200214    LM308  2020/02/14   
215  ssM-CpGiant-5-20200214    LM309  2020/02/14   
216  ssM-CpGiant-6-20200214    LM310  2020/02/14   

                                  杂交探针  杂交时间h  PostPCR循环数 PostPCR浓度ng/ul  \
0    SeqCap Epi CpGiant Enrichment Kit   66.5          12           16.2   
1    SeqCap Epi CpGiant Enrichment Kit   67.5          13           17.6   
2    SeqCap Epi CpGiant Enrichment Kit   67.0          12            110   
3    SeqCap Epi CpGiant Enrichment Kit   